<a href="https://colab.research.google.com/github/trefftzc/cis654/blob/main/Exploring_the_RSA_keys.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Finding the two prime factors of a large number is time consuming...

The purpose of this notebook is to illustrate that finding the two prime factors of a large number is time consuming.

These are some large prime numbers, close to 1,000,000.

999683
999721
999727
999749
999763
999769
999773
999809
999853
999863
999883
999907
999917
999931
999953
999959
999961
999979
999983

Taken from https://www.mathsisfun.com/numbers/prime-number-lists.html

These number take close to 20 bits to represent in binary.
Their product will be around 40 bits long.
That was the maximum length of a key back in the 90s in the early days of Netscape.

https://en.wikipedia.org/wiki/Crypto_Wars:

"Encryption export controls became a matter of public concern with the introduction of the personal computer. Phil Zimmermann's PGP cryptosystem and its distribution on the Internet in 1991 was the first major 'individual level' challenge to controls on export of cryptography. The growth of electronic commerce in the 1990s created additional pressure for reduced restrictions.[4] Shortly afterward, Netscape's SSL technology was widely adopted as a method for protecting credit card transactions using public key cryptography.

SSL-encrypted messages used the RC4 cipher, and used 128-bit keys. U.S. government export regulations would not permit crypto systems using 128-bit keys to be exported.[5] At this stage Western governments had, in practice, a split personality when it came to encryption; policy was made by the military cryptanalysts, who were solely concerned with preventing their 'enemies' acquiring secrets, but that policy was then communicated to commerce by officials whose job was to support industry.

The longest key size allowed for export without individual license proceedings was 40 bits, so Netscape developed two versions of its web browser. The "U.S. edition" had the full 128-bit strength. The "International Edition" had its effective key length reduced to 40 bits by revealing 88 bits of the key in the SSL protocol. Acquiring the 'U.S. domestic' version turned out to be sufficient hassle that most computer users, even in the U.S., ended up with the 'International' version,[6] whose weak 40-bit encryption could be broken in a matter of days using a single personal computer. A similar situation occurred with Lotus Notes for the same reasons.[7]"

In [1]:
prime1 = int(input("Enter one of the prime numbers above: "))
prime2 = int(input("Enter another one of the prime numbers above: "))
product = prime1 * prime2
print("Their product is: ",product)

Enter one of the prime numbers above:  999769
Enter another one of the prime numbers above: 999959
Their product is:  999728009471


In [2]:
import math
# Calculating the square root of the product
square_root = math.sqrt(product)
print("The square root is: ",square_root)

The square root is:  999863.9954868862


In [3]:
# Now, a brute force approach to find the two factors:
int_square_root = int(square_root)
for i in range(2,int_square_root):
  if product % i == 0:
    print("Found the factors: ",i," and ",product//i)
    break;

Found the factors:  999769  and  999959


Once the two prime factors of the RSA modulus are known, along with the exponent e, it is possible to calculate the value of d.

To complete the exercise, let's calculate the exponent e
that would be used along with the product of the primes (rsa-modulus
)
as the public key:

In [25]:
#Importing the greatest common divisor method from math
from math import gcd

#Calculate and display the Euler’s totient.
RSA_modulus = product
totient = (prime1 - 1)*(prime2 -1)
print("  Euler's totient -----> " + str(totient))

#Choosing the public-key exponent
public_exponent = 0



for e in range(3, totient-1):
  if gcd(e, totient) == 1:
    public_exponent = e
    break


#Display the public-key exponent e
print("  Public-Key exponent, e -----> " + str(public_exponent))

#Display the public key
print("  Public Key -----> (" + str(public_exponent) + ", " + str(RSA_modulus) + ")")



  Euler's totient -----> 999726009744
  Public-Key exponent, e -----> 5
  Public Key -----> (5, 999728009471)


Let's use that public key to encrypt one character.

In [26]:
#ENCRYPTION
#Plain text setup
print(" For the sake of simplicity, we are going to encrypt one character. Please enter below one character only. ")
plain_text = str(input(" Please enter the character that you would want to encrypt: "))

#Using ord to get ASCII encoding of the character entered
#chr is used to generate a character from an ASCII encoding

cipher_text = ((ord(plain_text)**public_exponent) % RSA_modulus)

print("  Plain Text " + plain_text + " encrypted to " + str(cipher_text))

 For the sake of simplicity, we are going to encrypt one character. Please enter below one character only. 
 Please enter the character that you would want to encrypt: a
  Plain Text a encrypted to 8587340257


So, given a public key, and an encrypted message, we can decrypt the message by:
1. Finding the prime factors of the RSA_modulus
2. Calculating the private key's exponent d
3. Applying the decryption function


In [20]:
#The following two functions will return a value of d when you pass it the parameters public-key exponent and totient.
def extended_gcd(aa, bb):
    lastremainder, remainder = abs(aa), abs(bb)
    x, lastx, y, lasty = 0, 1, 1, 0
    while remainder:
        lastremainder, (quotient, remainder) = remainder, divmod(lastremainder, remainder)
        x, lastx = lastx - quotient*x, x
        y, lasty = lasty - quotient*y, y
    return lastremainder, lastx * (-1 if aa < 0 else 1), lasty * (-1 if bb < 0 else 1)

#We produce the private-key exponent by finding the modular inverse of the public-key exponent, using the totient as the modulus.
def modinv(a, m):
	g, x, y = extended_gcd(a, m)
	if g != 1:
		raise ValueError
	return x % m

  #Find the modular inverse of the public-key exponent and use as the private-key exponent
private_exponent = modinv(public_exponent, totient)

#Display the private-key exponent e
print("  Private-Key exponent, d -----> " + str(private_exponent))

#Display the private key
print("  Private Key -----> (" + str(private_exponent) + ", " + str(RSA_modulus) + ")")

  Private-Key exponent, d -----> 106323068897
  Private Key -----> (106323068897, 999728009471)


Decrypting with this very large exponent would take too long.

Let's consider a simpler example.

We are given the following public key and encrypted message:

Public Key -----> (5,906527)

The encrypted value is 716513

So to find the original message, we follow the same three previous steps:

1. Find the prime factors of the RSA_modulus of the public key



In [33]:
RSA_modulus = 906527
square_root = math.sqrt(906527) # 906257 is the RSA modulus of the public key
int_square_root = int(square_root)
for i in range(2,int_square_root):
  if RSA_modulus % i == 0:
    print("Found the factors: ",i," and ",RSA_modulus//i)
    break;
prime1 = i
prime2 = RSA_modulus//i

Found the factors:  239  and  3793


2. Calculate the private exponent:

In [34]:
totient = (prime1 - 1)*(prime2 - 1)
public_exponent = 5
#Find the modular inverse of the public-key exponent and use as the private-key exponent
private_exponent = modinv(public_exponent, totient)

#Display the private-key exponent e
print("  Private-Key exponent, d -----> " + str(private_exponent))

#Display the private key
print("  Private Key -----> (" + str(private_exponent) + ", " + str(RSA_modulus) + ")")

  Private-Key exponent, d -----> 721997
  Private Key -----> (721997, 906527)


3. Decrypt

In [40]:
#DECRYPTION
cipher_text = 716513
message = chr(((cipher_text)**private_exponent) % RSA_modulus)
print((str(cipher_text) + " decrypted to " + str(message)))

716513 decrypted to a
